In [1]:
from os import listdir
import pandas as pd

In [17]:
MISSING = " "

# Load Data

In [2]:
files = listdir("../chart")

dfs_chart = {}
for f in files:
    year = f.split("_")[0]
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [3]:
dfs_chart["2006"]["Janvier"]

,Rank,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,Juanes,La Camisa Negra,Juanes,"9 août 1972 (50 ans)Medellin, Colombie",,,,
1,2,Madonna,Hung Up,Madonna,"16 août 1958 (64 ans)Bay City, Michigan (États...",,,Américaine,
2,3,Johnny Hallyday,Mon Plus Beau Noël,Johnny Hallyday,15 juin 1943 Paris 9e (France),,,Française,
3,4,Tina Arena,Aimer Jusqu'à L'impossible,Tina Arena,"1er novembre 1967 (55 ans)Melbourne, Australie",,,,
4,5,Star Academy,Santiano,Star Academy (France),,,,,France
...,...,...,...,...,...,...,...,...,...
95,96,Jenifer,Serre-moi,Jenifer,15 novembre 1982 (40 ans)Nice,,,,
96,97,Lucie Bernardoni,"Petit Rat, Petit Loup",Lucie Bernardoni,"4 février 1987 (35 ans)Nice, Alpes-Maritimes, ...",,,,
97,98,Ilona Mitrecey,C'est Les Vacances,Ilona Mitrecey,1er septembre 1993 (29 ans)Fontenay-aux-Roses ...,,,,
98,99,Kanye West,Gold Digger,Kanye West,"8 juin 1977 (45 ans)Atlanta, Géorgie (États-Unis)",,,,


In [7]:
artist = pd.read_csv("../artist.csv", index_col=0)

In [8]:
artist

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
Artist,,,,,,
O.T. Genasis,O.T. Genasis,"18 juin 1987 (35 ans)Atlanta, Géorgie, États-...",,,,
Angie Be,Angie Be,1977,,,,
The Merrymakers,Superstar,,,,,
Kids Will Rock You 2,Will Smith,"25 septembre 1968 (54 ans)Philadelphie, Pennsy...",,,américaine,
Cerena,Cérena,20 juin 1981 (41 ans)Bastia (Haute-Corse),,,,
...,...,...,...,...,...,...
ATB,ATB (musicien),"26 février 1973 (49 ans)[2]Freiberg, Allemagne",,,,
Sofia Carson,Sofia Carson,"10 avril 1993 (29 ans)Fort Lauderdale, Florid...",,,américaine,
Boy & Bear,Bear McCreary,17 février 1979 (43 ans)Fort Lauderdale,,,américaine,


# Chart analysis

In [34]:
dfs = []
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs.append(dfs_chart[k1][k2])

chart_merged = pd.concat(dfs)
chart_merged.drop(columns=["Rank"], inplace=True)
chart_merged.drop_duplicates(inplace=True)

In [35]:
chart_merged["Missing"] = (
    (chart_merged["Naissance"] == MISSING) &
    (chart_merged["Pays d'origine"] == MISSING) &
    (chart_merged["Origine"] == MISSING) &
    (chart_merged["Nationalité"] == MISSING) &
    (chart_merged["Pays"] == MISSING)
)

In [38]:
nb_title_missing_artist = chart_merged.groupby("Artist")["Missing"].sum().sort_values(ascending=False)

In [40]:
nb_title_missing_artist[nb_title_missing_artist > 2]

Artist
Landy                    12
Eva                      11
Swedish House Mafia       6
Pigloo                    5
Cauet                     4
Basto!                    4
Dry                       4
Titou Le Lapinou          4
Charlotte Aux Fraises     3
Anaklein                  3
Amadou & Mariam           3
T-rio                     3
Lynda                     3
Sasso                     3
The Rolling Stones        3
Larry                     3
Name: Missing, dtype: int64

In [37]:
chart_merged[chart_merged["Artist"] == "Landy"]

,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,Missing
63,Landy,Aucune Limite,Landy,,,,,,True
92,Landy,Toi T'es Chelou,Landy,,,,,,True
7,Landy,Enfants Terribles,Landy,,,,,,True
11,Landy,Millions D'euros,Landy,,,,,,True
45,Landy,Médusa,Landy,,,,,,True
47,Landy,Ma Werss,Landy,,,,,,True
71,Landy,V12,Landy,,,,,,True
78,Landy,Prends Ta Paye,Landy,,,,,,True
83,Landy,Filon,Landy,,,,,,True
85,Landy,Le Même,Landy,,,,,,True


# Artist analysis

In [9]:
cols = artist.columns.tolist()
cols.remove("Naissance")
cols.remove("Artist_wiki")

for col in cols:
    print(artist[col].value_counts(), "\n\n")

                      1886
france                 117
états-unis             114
royaume-unis            61
allemagne               19
canada                   9
australie                8
pays-bas                 8
suède                    5
espagne                  4
italie                   4
irlande                  3
moldavie                 3
angleterre               3
finlande                 3
danemark                 2
roumanie                 2
islande                  2
corée du sud             2
belgique                 2
détroit  michigan        1
autriche                 1
seychelles               1
russie                   1
écosse                   1
(miami, floride)         1
suisse                   1
norvège                  1
jamaïque                 1
ghana                    1
côte d'ivoire            1
hongrie                  1
japon                    1
jönköping  (suède)       1
afrique du sud           1
Name: Pays d'origine, dtype: int64 


                 

In [10]:
artist["Naissance"].str.lower().value_counts()[artist["Naissance"].str.lower().value_counts() > 2]

                                                    946
15 avril 1997 (25 ans)paris 14e                       4
19 avril 1985 (37 ans)paris (20e),  france            3
21 mars 1987 (35 ans)marseille, bouches-du-rhône      3
20 avril 1973 (49 ans)turin, piémont,  italie         3
1er mai 1980 (42 ans)chambray-lès-tours, france       3
3 avril 2000 (22 ans)[1]saint-denis, france           3
Name: Naissance, dtype: int64

In [11]:
artist[artist["Naissance"].str.lower().str.contains("paris 14e")]

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
Artist,,,,,,
Zkr,PLK (rappeur),15 avril 1997 (25 ans)Paris 14e,,,française,
Yaro,PLK (rappeur),15 avril 1997 (25 ans)Paris 14e,,,française,
Alpha Wann,Alpha Wann,2 juillet 1989 (33 ans)Paris 14e (France),,,,
Oboy,PLK (rappeur),15 avril 1997 (25 ans)Paris 14e,,,française,
Plk,PLK (rappeur),15 avril 1997 (25 ans)Paris 14e,,,française,
